# OpenML CC18 Metalearning Benchmark

In [1]:
%matplotlib inline

In [28]:
import pandas as pd
import pandera as pa
import plotly.express as px

## Load Data

In [10]:
JOB = 302


check_betw_zero_one = pa.Check(lambda s: (0 <= s) & (s <= 1))

INFERENCE_RESULT_SCHEMA = pa.DataFrameSchema(
    columns={
        "data_env": pa.Column(pa.String),
        "n_inference_steps": pa.Column(pa.Int),
        "is_valid": pa.Column(pa.Bool),
        "mlf_description": pa.Column(pa.String),
        "reward": pa.Column(pa.Float, check_betw_zero_one),
        "validation_score": pa.Column(pa.Float, check_betw_zero_one),
    },
    coerce=True,
)

@pa.check_output(INFERENCE_RESULT_SCHEMA)
def data_to_longform(inference_results):
    return (
        inference_results
        [["data_env", "n_inference_steps", "key", "value"]]
        .pivot_table(
            index=["data_env", "n_inference_steps"],
            columns="key",
            values="value",
            aggfunc=lambda x: x,
        )
        .reset_index()
        .dropna()
        .rename_axis(None, axis=1)
    )

In [29]:
inference_results = pd.concat([
    (
        pd.read_csv(f"../floyd_outputs/{JOB}/{env}_env_inference_results.csv")
        .pipe(data_to_longform)
        .assign(data_env_partition=env)
    ) for env in ["test", "training"]
])
inference_results.head()

,data_env,n_inference_steps,is_valid,mlf_description,reward,validation_score,data_env_partition
100,kaggle.homesite_quote_conversion,0,True,{'SimpleImputer': ColumnTransformer(n_jobs=Non...,0.954995,0.954995,test
201,kaggle.santander_customer_satisfaction,1,True,{'SimpleImputer': ColumnTransformer(n_jobs=Non...,0.818219,0.818219,test
202,kaggle.santander_customer_satisfaction,2,True,{'SimpleImputer': ColumnTransformer(n_jobs=Non...,0.803531,0.803531,test
203,kaggle.santander_customer_satisfaction,3,True,{'SimpleImputer': ColumnTransformer(n_jobs=Non...,0.804451,0.804451,test
204,kaggle.santander_customer_satisfaction,4,True,{'SimpleImputer': ColumnTransformer(n_jobs=Non...,0.807202,0.807202,test


## Plot Rewards

In [33]:
px.line(
    inference_results.query("data_env_partition == 'training'"),
    x="n_inference_steps",
    y="validation_score",
    template="plotly_white",
    color="data_env",
)

In [34]:
px.line(
    inference_results.query("data_env_partition == 'test'"),
    x="n_inference_steps",
    y="validation_score",
    template="plotly_white",
    color="data_env",
)